<i><b>Public AI</b></i>
<br>

###  &nbsp;&nbsp; **✎&nbsp;&nbsp;Week 2. CNN Basis**
# Section 3. 고차원에서의 합성곱 연산

### _Objective_
1. 대부분의 합성곱 연산은 4차원의 데이터를 다룹니다. 이를 어떻게 연산하는 지를 배워봅니다. <br>
2. 합성곱 연산의 연산량을 줄여주는 풀링 연산에 대해 배워봅니다.<br> 
  


In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Input, Conv2D
from tensorflow.keras.models import Model

<br><br>

# \[ 1. 컬러 이미지 데이터(3차원) 합성곱 연산 \]

----

----

> *이전까지는 2차원 이미지인 흑백 이미지를 다루는 합성곱 연산을 살펴봤습니다.*<br>
> 이번 시간에는 3차원 이상의 이미지를 Convolution 하는 방법을 알아 보겠습니다. 

## 1. 하나의 필터로 3차원 데이터 합성곱 진행하기
---

* 채널 방향으로 입력 데이터와 필터의 합성곱 연산을 수행하고, 그 결과 데이터를 더해 하나의 출력값으로 표현합니다.
* 아래의 3차원 데이터를 통해, 합성곱 연산이 어떤 식으로 진행되는지 살펴보도록 하겠습니다.

### (1) 3차원 데이터 구성하기

아래와 같이 3차원으로 구성된 입력 행렬을 구성하였습니다.<br>

In [2]:
in0 = np.array([
    [1,4,2,0],
    [2,3,1,0],
    [3,1,2,3],
    [4,3,2,1]
])
in1 = np.array([
    [1,7,2,1],
    [3,2,1,9],
    [0,1,5,1],
    [4,6,2,4]
])
in2 = np.array([
    [1,5,2,1],
    [3,2,1,9],
    [0,1,3,1],
    [4,6,2,7]
])
color_input = np.stack([in0, in1, in2],axis=-1)
print("입력값의 형태 (H, W, C) :({},{},{})".format(*color_input.shape))

입력값의 형태 (H, W, C) :(4,4,3)


### (2) Filter 구성하기

여기서 중요한 것은 입력값의 채널 수(C)와 필터의 채널 수(C)가 동일해야 한다는 것입니다.<br>
입력값의 채널에 따라 채널 별 필터가 존재하고, 각 채널 별로 합성곱 연산을 진행한 후, 하나의 값으로 출력됩니다.<br>

In [3]:
f0 = np.array([
    [2,0,1],
    [0,1,2],
    [1,0,2]
])
f1 = np.array([
    [0,1,3],
    [2,1,3],
    [4,1,2]
])
f2 = np.array([
    [3,2,1],
    [2,2,3],
    [0,0,1]
])
filters = np.stack([f0,f1,f2],axis=-1)
print("Filter의 형태 (H, W, C) : ({},{},{})".format(*filters.shape))

Filter의 형태 (H, W, C) : (3,3,3)


### (3) 합성곱 연산 진행하기

padding은 valid, stride은 1로 두고 연산하면 아래와 같습니다.
![Imgur](https://i.imgur.com/ovXS3UT.png)

In [4]:
# filters를 필터로하는 color_input과 filters간의 합성곱 연산을 수행해주세요
# fix me!

(0,0) 번째 값 : 82
(0,1) 번째 값 : 118
(1,0) 번째 값 : 92
(1,1) 번째 값 : 126


array([[ 82., 118.],
       [ 92., 126.]])

### (4) Keras 로 합성곱 연산 진행하기 

In [5]:
# color_input과 filters간의 convolution 연산을 keras를 사용해 진행해주세요
# fix me!

array([[[[ 82.],
         [118.]],

        [[ 92.],
         [126.]]]], dtype=float32)

### (5) 블록으로 시각화하기

블록으로 시각화한다면, 아래와 같이 나타낼 수 있습니다.

![Imgur](https://i.imgur.com/uNUYhhx.png)

* 입력 특징맵의 형태 : $(h_{in},w_{in},c_{in})$
* 필터의 형태 : $(h_f,w_f,c_{in})$
* 출력 특징맵의 형태 : $(h_{out},w_{out})$

출력의 형태는 이전에 배운 것과 마찬가지로<br>
필터의 크기, Stride, padding으로 결정됩니다.

## 2. 복수개의 필터로 3차원 데이터 합성곱 진행하기
---

* 하나의 Filter은 하나의 특징을 추출할 수 있습니다. <br>
* 영상을 파악하기 위해서는 복수개의 특징을 함께 추출해, 판단해야 합니다.<br>
* 복수개의 필터는 아래와 같이 진행됩니다.

### (1) Filter 구성하기

하나의 필터는 $(h_f,w_f,c_{in})$의 형태로 구성됩니다.<br>
이러한 필터들을 복수개로 구성해 쌓으면 아래와 같습니다.

In [6]:
filter_1 = np.array([
    [[2,0,1],[0,1,2],[1,0,2]],
    [[0,1,3],[2,1,3],[4,1,2]],
    [[3,2,1],[2,2,3],[0,0,1]]])
filter_2 = np.array([
    [[4,0,1],[0,0,4],[0,3,2]],
    [[6,1,2],[3,5,1],[2,3,2]],
    [[1,4,1],[1,3,1],[2,1,0]]])

    
filters = np.stack([filter_1,filter_2],axis=0)
print("Filter의 형태 (N, H, W, C) : ({},{},{},{})".format(*filters.shape))

Filter의 형태 (N, H, W, C) : (2,3,3,3)


In [7]:
# filters를 필터로하는 color_input과 filters간의 합성곱 연산을 수행해주세요
# fix me!

array([[[ 81., 115.],
        [ 64., 204.]],

       [[ 89., 151.],
        [ 76., 167.]]])

### (2) Keras 로 합성곱 연산 진행하기 

![Imgur](https://i.imgur.com/Q1wXw64.png)

In [8]:
# color_input과 filters간의 convolution 연산을 keras를 사용해 진행해주세요
# filter의 갯수에 유의해주세요!
# fix me!

array([[[[ 99.,  86.],
         [155.,  91.]],

        [[164.,  87.],
         [156., 141.]]]], dtype=float32)

### (3) 블록으로 시각화하기

블록으로 시각화한다면, 아래와 같이 나타낼 수 있습니다.


![Imgur](https://i.imgur.com/ONua9B4.png)




* 입력 특징맵의 형태 : $(h_{in},w_{in},c_{in})$
* 필터의 형태 : $(n_f,h_f,w_f,c_{in})$
* 출력 특징맵의 형태 : $(h_{out},w_{out},n_f)$

## 3. 복수개의 필터로 배치단위 데이터 합성곱 진행하기
---

* 신경망을 학습시킬 때에는 단일 데이터보다 보통 배치 단위로 처리합니다.<br>
* 배치 단위로 진행될 경우 어떤 식으로 데이터의 형태가 변화하는지를 파악하는데 초점을 두도록 하겠습니다.

![Imgur](https://i.imgur.com/BlvHzCu.png)

* 입력 특징맵의 형태 : $(n_{batch}, h_{in},w_{in},c_{in})$
* 필터의 형태 : $(n_f,h_f,w_f,c_{in})$
* 출력 특징맵의 형태 : $(n_{batch}, h_{out},w_{out},n_f)$

---
⊙ Copyright(c) 2020 by PublicAI. All rights reserved <br>
All pictures, codes, writings cannot be copied without permission. <br>
Writen by PAI(info@publicai.co.kr) <br>
last updated on 2020/01/4 <br>

---